In [1]:
import remi.gui as gui
from remi import start, App
import time,threading
import qrcode,base64,io,cv2
import gradio as gr
import BaseDeploy as bd
import matplotlib.pyplot as plt
import random

model_path = 'E:\\Lindia\\2023.08.21 AI Lab 夏令营\\视力检测\\out_file\\hand.onnx'
model = bd(model_path)

camera = cv2.VideoCapture(0)
# 临时变量，用于保存画面
tempcam = None
# 图片名称编号
f_num = 1

#全局变量控制进程
running=0

#视力表素材
E_path='E:\\Lindia\\2023.08.21 AI Lab 夏令营\\视力检测\\E.jpg'
E = plt.imread(E_path)
E = cv2.resize(E, (257,257), interpolation=cv2.INTER_LINEAR)
E = cv2.imencode('.jpg', E)[1] #解码成图像格式
base64_data = str(base64.b64encode(E))[2:-1]
E='data:image/jpeg;base64,%s'%base64_data

#视力计算
'''
高度近视：0
中度近视：1
低度近视：2
矫正：3
不近视：4
'''
eye_list=[257,81,41,26]

#全局变量，保存目前测量结果
eyeSight=0

#全局变量，保存连续错误次数，当不连续正确或测量结果变化时清空
fail=0



# 返回摄像头画面，自定义大小
def get_frames(x,y):
    global tempcam
    success, frame = camera.read()  # read the camera frame
    if success:
        tempcam = frame #临时变量获取摄像头画面
        frame = cv2.flip(frame, 1, dst=None) #镜像
        frame = cv2.resize(frame, (x, y), interpolation=cv2.INTER_LINEAR) #自定义大小
        image = cv2.imencode('.jpg', frame)[1] #解码成图像格式
        base64_data = str(base64.b64encode(image))[2:-1]
        data='data:image/jpeg;base64,%s'%base64_data
        return data
    else:
        return get_img("摄像头启动失败")

# Web主程序
class MyApp(App):
    def __init__(self, *args):
        super(MyApp, self).__init__(*args)

    def main(self):
        #页面大小
        container = gui.VBox(width=700, height=450,style={'margin':'0px auto'}) 
        
        
        #页面元素
        bts = gui.HBox(width=300, height=50)
        self.bt_start = gui.Button('[ Start ]')
        self.bt_end = gui.Button('[ End ]')
        bts.append(self.bt_start)
        bts.append(self.bt_end)
        
        self.E=gui.Image(E)
        
        pic = gui.HBox(width=350, height=400)
        self.stream = gui.Image(get_frames(150,100)) #实施显示摄像头画面
        self.photo = gui.Image(get_frames(150,100)) #显示拍摄照片        
        pic.append(self.stream)
        pic.append(self.photo)
        
        self.lbl_01 = gui.Label('Web摄像头画面采集器',style={'font-size': '25px'}) #标题
        self.lbl_running = gui.Label('Stopped')
        self.result = gui.Label('Result will show here..') #预测结果

        
        # 添加到网页上
        container.append(self.lbl_01)
        container.append(self.E)
        container.append(pic)
        container.append(self.lbl_running)
        container.append(self.result)        
        container.append(bts)
        
        # 按钮按下时执行
        self.bt_start.onclick.do(self.on_button_pressed,'start')
        self.bt_end.onclick.do(self.on_button_pressed,'end')
        
        # 开启新的进程刷新画面
        stream = threading.Thread(target=self.showimg)
        stream.start()
        
        # 开启新的进程自动拍照
        capture = threading.Thread(target=self.capture)
        capture.start()
        

        # returning the root widget
        return container
    
    #-------------------------------------------------------------------------
    def on_button_pressed(self, emitter, tag):
        global running
        if tag=='start':
            running=1
            self.lbl_running.set_text("Running..")
            print(running)
        if tag=='end':
            running=0
            self.lbl_running.set_text("Stopped")
            print(running)
    
    def capture(self):
        global eyeSight
        global fail
        while True:
            if not running:
                continue
            this_E=E = plt.imread(E_path)
            if eyeSight==0:
                this_E = cv2.resize(this_E, (257,257), interpolation=cv2.INTER_LINEAR)
            if eyeSight==1:
                this_E = cv2.resize(this_E, (81,81), interpolation=cv2.INTER_LINEAR)            
            if eyeSight==2:
                this_E = cv2.resize(this_E, (47,47), interpolation=cv2.INTER_LINEAR)            
            if eyeSight==3:
                this_E = cv2.resize(this_E, (26,26), interpolation=cv2.INTER_LINEAR)            
            
            size=eye_list[eyeSight]
            this_E = cv2.resize(this_E, (size,size), interpolation=cv2.INTER_LINEAR)
            #随机方向
            direction=random.randint(0,3)
            if direction==0:
                ans='right'
            if direction==1:
                ans='left'
                this_E = cv2.flip(this_E, 1, dst=None)
            if direction==2:
                ans='down'
                this_E = cv2.rotate(this_E, cv2.ROTATE_90_CLOCKWISE)
            if direction==3:
                ans='up'
                this_E = cv2.rotate(this_E, cv2.ROTATE_90_COUNTERCLOCKWISE)   
                
            this_E = cv2.imencode('.jpg', this_E)[1] #解码成图像格式
            base64_data = str(base64.b64encode(this_E))[2:-1]
            this_E='data:image/jpeg;base64,%s'%base64_data
            self.E.set_image(this_E)
            
            time.sleep(5)
            test = tempcam
            test = cv2.flip(test, 1, dst=None)
            frame = test
            frame = cv2.resize(frame, (150,100), interpolation=cv2.INTER_LINEAR) #自定义大小
            image = cv2.imencode('.jpg', frame)[1] #解码成图像格式
            base64_data = str(base64.b64encode(image))[2:-1]
            data='data:image/jpeg;base64,%s'%base64_data
            self.photo.set_image(data)
            result = model.inference(test)
            result = model.print_result(result)
            print(result)
            if result['预测结果']==ans:
                self.result.set_text('True')
            else:
                self.result.set_text('False')


    
    # 刷新网页的摄像头画面
    def showimg(self):
        while True:
            if not running:
                continue                            
            self.stream.set_image(get_frames(150,100))
            time.sleep(0.2)

# starts the web server
start(MyApp,title='摄像头实时视频',address='0.0.0.0')


Success load model info generate by MMEdu>=0.1.15: {"codebase": "MMCls", "modelname": "LeNet", "classes": ["down", "left", "right", "up"]}


remi.server      INFO     Started httpserver http://0.0.0.0:52861/
remi.request     INFO     built UI (path=/)
127.0.0.1 - - [25/Aug/2023 13:56:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2023 13:56:20] "GET /res:style.css HTTP/1.1" 200 -
remi.server.ws   INFO     connection established: ('127.0.0.1', 52867)
remi.server.ws   INFO     handshake complete
127.0.0.1 - - [25/Aug/2023 13:56:20] "GET /res:font.woff2 HTTP/1.1" 200 -


1
{'标签': 3, '置信度': 0.9139211, '预测结果': 'up'}
{'标签': 1, '置信度': 0.5780173, '预测结果': 'left'}
{'标签': 3, '置信度': 0.9164292, '预测结果': 'up'}
{'标签': 1, '置信度': 0.87617785, '预测结果': 'left'}
{'标签': 1, '置信度': 0.8347247, '预测结果': 'left'}
{'标签': 1, '置信度': 0.7942498, '预测结果': 'left'}
{'标签': 3, '置信度': 0.9164322, '预测结果': 'up'}
{'标签': 3, '置信度': 0.8839384, '预测结果': 'up'}
{'标签': 1, '置信度': 0.65537065, '预测结果': 'left'}
{'标签': 3, '置信度': 0.6433255, '预测结果': 'up'}
{'标签': 2, '置信度': 0.5971127, '预测结果': 'right'}
{'标签': 0, '置信度': 0.68501854, '预测结果': 'down'}
{'标签': 2, '置信度': 0.5478245, '预测结果': 'right'}
{'标签': 3, '置信度': 0.92097735, '预测结果': 'up'}
{'标签': 2, '置信度': 0.98548836, '预测结果': 'right'}
{'标签': 2, '置信度': 0.9731705, '预测结果': 'right'}
{'标签': 2, '置信度': 0.8542363, '预测结果': 'right'}
{'标签': 0, '置信度': 0.69228995, '预测结果': 'down'}
{'标签': 3, '置信度': 0.95229214, '预测结果': 'up'}
{'标签': 3, '置信度': 0.95608103, '预测结果': 'up'}
{'标签': 2, '置信度': 0.9745205, '预测结果': 'right'}
{'标签': 3, '置信度': 0.9058182, '预测结果': 'up'}
{'标签': 3, '置信度': 0.8148278, '预测结果'

remi.server      INFO     *** signal 2 received.
